# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

## P2

In [ ]:
def sinplot(flip=1):
    x = np.linspace(0, 14, 100)
    for i in range(1, 7):
        plt.plot(x, np.sin(x + i * .5) * (7 - i) * flip)
sinplot()

In [ ]:
def estilo(sty = "whitegrid"):
    param = {'axes.facecolor': 'white',
             'axes.edgecolor': 'black',
             'axes.grid': True,
             'axes.axisbelow': 'line',
             'axes.labelcolor': 'black',
             'figure.facecolor': (1, 1, 1, 0),
             'grid.color': '#b0b0b0',
             'grid.linestyle': '-',
             'text.color': 'black',
             'xtick.color': 'red',
             'ytick.color': 'black',
             'xtick.direction': 'out',
             'ytick.direction': 'out',
             'lines.solid_capstyle': 'projecting',
             'patch.edgecolor': 'black',
             'image.cmap': 'viridis',
             'font.family': ['sans-serif'],
             'font.sans-serif': ['DejaVu Sans',
             'Bitstream Vera Sans',
             'Computer Modern Sans Serif',
             'Lucida Grande',
             'Verdana',
             'Geneva',
             'Lucid',
             'Arial',
             'Helvetica',
             'Avant Garde',
             'sans-serif'],
             'patch.force_edgecolor': False,
             'xtick.bottom': True,
             'xtick.top': False,
             'ytick.left': True,
             'ytick.right': False,
             'axes.spines.left': True,
             'axes.spines.bottom': True,
             'axes.spines.right': False,
             'axes.spines.top': False}
    sns.set_style(sty, param)

In [ ]:
estilo()
sinplot()

## P3

In [5]:
from sklearn.base import BaseEstimator, RegressorMixin

In [6]:
# 1
class RegresionBayesianaEmpirica(BaseEstimator, RegressorMixin):

    def __init__(self, alpha_0, beta_0, tol=1e-5, maxiter=200):
        self.alpha_0 = alpha_0
        self.beta_0 = beta_0
        self.tol = tol
        self.maxiter = maxiter

    def get_posteriori(self, X, y, alpha, beta):
        N,d = X.shape
        S_N_1 = alpha*np.identity(d) + beta*(X.T @ X)
        S_N = np.linalg.inv(S_N_1)
        m_N = beta*((S_N @ X.T) @ y)
        return m_N, S_N

    def fit(self, X, y):
        N,d = X.shape
        alpha_old = self.alpha_0
        beta_old = self.beta_0
        print('Iteracion: 1')
        print('valores actuales, alpha:{}, beta:{}'.format(round(alpha_old,6),round(beta_old,6)))
        lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
        gamma = np.sum(lambdas/(alpha_old + lambdas))
        m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
        alpha_new = gamma/(m_N.T @ m_N)
        beta_new = (N-gamma)/np.sum((y-m_N.T@X.T)**2)
        for k in range(2, self.maxiter+1): 
            err_alpha = abs(alpha_old - alpha_new)
            err_beta = abs(beta_old - beta_new)
            if (err_alpha < self.tol) and (err_beta < self.tol):
                break
            else:
                print('Iteracion: ', k)
                print('valores actuales, alpha:{}, beta:{}'.format(round(alpha_new,6),round(beta_new,6)))
                alpha_old = alpha_new
                beta_old = beta_new
                lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
                gamma = np.sum(lambdas/(alpha_old + lambdas))
                m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
                alpha_new = gamma/(m_N.T @ m_N)
                beta_new = (N-gamma)/np.sum((y-m_N.T@X.T)**2)
        self.alpha_0 = alpha_new
        self.beta_0 = beta_new
        print('\nvalores finales, alpha:{}, beta:{}'.format(round(alpha_new,6),round(beta_new,6)))
        m_N, S_N = self.get_posteriori(X,y,alpha_new,beta_new)
        self.m_N = m_N
        self.S_N = S_N

    def predict(self, X_, return_std=False):
        y_ = X_ @ self.m_N
        y_std = 1/self.beta_0 + np.diag((X_ @ self.S_N) @ X_.T)
        if return_std:
            return y_, y_std
        else:
            return y_    

In [ ]:
# ejemplito del fit
mat = np.array([[1, 0, 0], [0, 1, 0], [0,0,1], [0,0,0]])
mat_y = np.array([10, 12,7,5])
rbe = RegresionBayesianaEmpirica(0.5, 2/3)
rbe.fit(mat, mat_y)
print('prediccion: ',rbe.predict(np.array([[1,2,3]]))) 

In [8]:
# 2
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
def data_1d(df_column):
    return df_column.values.reshape([-1,1])
#reshape
rs = FunctionTransformer(data_1d)

# categorical pipeline
cat_pipe = Pipeline([('reshape', rs), ('ordinal', OneHotEncoder(handle_unknown='ignore'))])

# numerical pipeline
num_pipe = Pipeline([('reshape', rs), ('scaler', StandarScaler()), ('poly', PolynomialFeatures(degree=3))])

In [ ]:
## JS: primero hay que escoger/definir las columnas numericas y categoricas
# num_cols = ['price', 'surface', 'densidad']
# cat_cols =['n_rooms']
prep = ColumnTransformer(transformers=[('num', num_pipe, num_cols), ('cat', cat_pipe, cat_cols)])

In [ ]:
# 3... y 4?
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2)
alpha_0 = 1e-7 
beta_0 =  1e-5
el_pipe = Pipeline([('prep', prep), ('classifier', RegresionBayesianaEmpirica(alpha_0,beta_0))])
el_pipe.fit(X_train, y_train)
y_pred = el_pipe.predict(X_test)
RMSE = np.sqrt(np.mean((y_pred - y_test)**2))
R2 = el_pipe.score(X_test, y_test)
print('RMSE: ', RMSE)
print('R^2: ', R2)

In [ ]:
import pickle
# save the model to disk
filename = 'modelo.pkl'
pickle.dump(el_pipe, open(filename, 'wb'))

# some time later... 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [ ]:
# 5
from sklearn.linear_model import BayesianRidge

ridge_pipe = Pipeline([('prep', prep), ('classifier', BayesianRidge())])
ridge_pipe.fit(X_train, y_train)
y_pred_ridge = ridge_pipe.predict(X_test)
RMSE_ridge = np.sqrt(np.mean((y_pred_ridge - y_test)**2))
R2_ridge = ridge_pipe.score(X_test, y_test)
print('RMSE ridge: ', RMSE_ridge)
print('R^2 ridge: ', R2_ridge)